In [381]:
import numpy as np

In [373]:
def P2R(radii, angle, round_num: bool = True, round_dec=3) -> complex:
    angle = angle * np.pi / 180
    return np.round(radii * (np.cos(angle) + 1j * np.sin(angle)), round_dec) if round_num == True else (radii * (np.cos(angle) + 1j * np.sin(angle)))

def R2P(x: complex, round_num = True, string: bool = False, sym: str = "", base: float = 1, round_dec = 3) -> list[float]|str:
    if not string:
        return [round(abs(x), round_dec), round(np.angle(x, deg = True), round_dec)] if round_num == True else [abs(x), np.angle(3)]
    else:
        return (
            f"{round(abs(x) * base, round_dec):>6} {sym} ∠ {round(np.angle(x, deg = True), round_dec):>6}"
            u"\N{DEGREE SIGN}"
        )

In [374]:
s_base: float = 100 * 10**6         # 100 MVA base
v_base: float = 400 * 10**3         # 400 kV base
z_base: float = v_base**2 / s_base  # Z = V^2 / S
i_base: float = s_base / v_base     # I = S / V

In [375]:
line_1_2: complex = (0.00 + (0.64j * 100)) / z_base     # impedance from bus 1 to bus 2
line_1_3: complex = (0.00 + (0.64j * 62.5)) / z_base    # impedance from bus 1 to bus 3
line_2_3: complex = (0.00 + (0.64j * 50)) / z_base      # impedance from bus 2 to bus 3

# Construct impedance matrix
imp_matrix: list[complex] = [
    line_1_2,
    line_1_3,
    line_2_3,
]

imp_matrix

[0.04j, 0.025j, 0.02j]

In [376]:
# Convert impedance to admittances
add_line_1_2: complex = 1 / line_1_2  
add_line_1_3: complex = 1 / line_1_3
add_line_2_3: complex = 1 / line_2_3

add_vector = [add_line_1_2, add_line_1_3, add_line_2_3]

# Create self admittances
Y_11: complex = add_line_1_2 + add_line_1_3
Y_22: complex = add_line_1_2 + add_line_2_3
Y_33: complex = add_line_1_3 + add_line_2_3

# Create mutual admittances
Y_12: complex = -add_line_1_2
Y_13: complex = -add_line_1_3

Y_21: complex = -add_line_1_2
Y_23: complex = -add_line_2_3

Y_31: complex = -add_line_1_3
Y_32: complex = -add_line_2_3

# Construct admittance matrix
admittance_matrix: np.ndarray = np.array(
    [
        [Y_11, Y_12, Y_13],
        [Y_21, Y_22, Y_23],
        [Y_31, Y_32, Y_33]
    ]
)

print("Admittance Matrix")
for i in admittance_matrix:
    print(f"{"|":^1} {i[0].imag:>6.2f}j {i[1].imag:>6.2f}j {i[2].imag:>6.2f}j {"|":^1}")

Admittance Matrix
| -65.00j  25.00j  40.00j |
|  25.00j -75.00j  50.00j |
|  40.00j  50.00j -90.00j |


In [377]:
emf_1: complex = P2R(1.000, 0.000, round_num=False)   # EMF at bus 1
emf_2: complex = P2R(0.986, -1.799, round_num=False)  # EMF at bus 2
emf_3: complex = P2R(1.000, -0.345, round_num=False)  # EMF at bus 3

# Create EMF matrix
emf_matrix = np.array(
    [
        [emf_1],
        [emf_2],
        [emf_3]
    ]
)

# Print out EMF values in rectangular form
print(f"{"|":^1} {"Bus":^6} {"|":^1} {"EMF":^19} {"|":^1}")
print(f"{"|":^1} {"------"} {"|":^1} {"-------------------"} {"|":^1}")
for i in range(len(emf_matrix)):
    neg_sign_real = False if emf_matrix[i][0].real >= 0 else True
    neg_sign_imag = False if emf_matrix[i][0].imag >= 0 else True
    print(f"{"|":1} {i + 1:^6} {"|":^1} "
        f"{"-" if neg_sign_real == True else "+":^3}"
        f"{abs(emf_matrix[i][0].real):^4.3f}"
        f"{"-" if neg_sign_imag == True else "+":^3}"
        f"{abs(emf_matrix[i][0].imag):^4.3f}"
        f"{"j V":^3}"
        f" {"|":^1}"
    )

|  Bus   |         EMF         |
| ------ | ------------------- |
|   1    |  + 1.000 + 0.000j V |
|   2    |  + 0.986 - 0.031j V |
|   3    |  + 1.000 - 0.006j V |


In [378]:
# Calculate branch currents
branch_currents = {
    "I_12": add_line_1_2 * (emf_1 - emf_2),
    "I_13": add_line_1_3 * (emf_1 - emf_3),
    "I_21": add_line_1_2 * (emf_2 - emf_1),
    "I_23": add_line_2_3 * (emf_2 - emf_3),
    "I_31": add_line_1_3 * (emf_3 - emf_1),
    "I_32": add_line_2_3 * (emf_3 - emf_2),
}

# branch_currents = {
#     "I_12": (emf_1 - emf_2) / imp_matrix[0],
#     "I_13": (emf_1 - emf_3) / imp_matrix[1],
#     "I_21": (emf_2 - emf_1) / imp_matrix[0],
#     "I_23": (emf_2 - emf_3) / imp_matrix[2],
#     "I_31": (emf_3 - emf_1) / imp_matrix[1],
#     "I_32": (emf_3 - emf_2) / imp_matrix[2],
# }


# Print branch currents in PU
print("Branch Currents in PU")
for key, value in branch_currents.items():
    print(f"{key}: {R2P(value, string = True, round_dec=5)}")
print("\n")

# Print branch currents in Amperes
print("Branch Currents in Amps")
for key, value in branch_currents.items():
    print(f"{key}: {R2P(value, string = True, sym = "A", base = i_base)}")

Branch Currents in PU
I_12: 0.85439  ∠ -25.079°
I_13: 0.24086  ∠ -0.1725°
I_21: 0.85439  ∠ 154.921°
I_23: 1.44131  ∠ 149.87417°
I_31: 0.24086  ∠ 179.8275°
I_32: 1.44131  ∠ -30.12583°


Branch Currents in Amps
I_12: 213.598 A ∠ -25.079°
I_13: 60.214 A ∠ -0.172°
I_21: 213.598 A ∠ 154.921°
I_23: 360.327 A ∠ 149.874°
I_31: 60.214 A ∠ 179.828°
I_32: 360.327 A ∠ -30.126°


In [379]:
# Branch power flows
apparent_power = {
    "S_12": emf_matrix[0][0] * np.conj(branch_currents["I_12"]),
    "S_13": emf_matrix[0][0] * np.conj(branch_currents["I_13"]),
    "S_21": emf_matrix[1][0] * np.conj(branch_currents["I_21"]),
    "S_23": emf_matrix[1][0] * np.conj(branch_currents["I_23"]),
    "S_31": emf_matrix[2][0] * np.conj(branch_currents["I_31"]),
    "S_32": emf_matrix[2][0] * np.conj(branch_currents["I_32"]),
}

print("Branch Power Flows in MVA")
for key, value in apparent_power.items():
    print(f"{key}: {R2P(value, string = True, sym = "MVA", base = s_base / 10**6)}")
print("\n")

print("Branch Power Flows in MW and MVAr")
for key, value in apparent_power.items():
    print(f"{key}: {round(value.real * 100, 3):>8} {"MW &":^5} {round(value.imag * 100, 3):>8} MVAr")

Branch Power Flows in MVA
S_12: 85.439 MVA ∠ 25.079°
S_13: 24.086 MVA ∠  0.172°
S_21: 84.243 MVA ∠ -156.72°
S_23: 142.113 MVA ∠ -151.673°
S_31: 24.086 MVA ∠ 179.827°
S_32: 144.131 MVA ∠ 29.781°


Branch Power Flows in MW and MVAr
S_12:   77.385 MW &    36.215 MVAr
S_13:   24.085 MW &     0.073 MVAr
S_21:  -77.385 MW &   -33.295 MVAr
S_23: -125.096 MW &   -67.433 MVAr
S_31:  -24.085 MW &     0.073 MVAr
S_32:  125.096 MW &    71.587 MVAr


In [380]:
bus_1 = P2R(1.000, 0.000, round_num=False)
bus_2 = P2R(0.986, -1.799, round_num=False)
bus_3 = P2R(1.000, -0.345, round_num=False)

vol_1_2_pu = R2P(bus_1 - bus_2, string = True)
vol_1_3_pu = R2P(bus_1 - bus_3, string = True)
vol_2_3_pu = R2P(bus_2 - bus_3, string = True)

vol_1_2 = R2P(bus_1 - bus_2, string = True, sym = "kV", base = v_base/1000)
vol_1_3 = R2P(bus_1 - bus_3, string = True, sym = "kV", base = v_base/1000)
vol_2_3 = R2P(bus_2 - bus_3, string = True, sym = "kV", base = v_base/1000)

vol_rec = [bus_1 - bus_2, bus_1 - bus_3, bus_2 - bus_3]

print(f"voltage 1 -> 2: {vol_1_2}")
print(f"voltage 1 -> 3: {vol_1_3}")
print(f"voltage 1 -> 2: {vol_2_3}")
print(f"\n")

print(f"voltage 1 -> 2: {vol_1_2_pu}")
print(f"voltage 1 -> 3: {vol_1_3_pu}")
print(f"voltage 1 -> 2: {vol_2_3_pu}")


voltage 1 -> 2:  13.67 kV ∠ 64.921°
voltage 1 -> 3:  2.409 kV ∠ 89.828°
voltage 1 -> 2:  11.53 kV ∠ -120.126°


voltage 1 -> 2:  0.034  ∠ 64.921°
voltage 1 -> 3:  0.006  ∠ 89.828°
voltage 1 -> 2:  0.029  ∠ -120.126°
